# Mixtral-8x7B-v0.1 benchmarks - single GPU

## Dependencies

In [ ]:
pip install --upgrade transformers

In [ ]:
pip install --upgrade safetensors

In [ ]:
pip install --upgrade accelerate

In [ ]:
pip install --upgrade bitsandbytes

In [ ]:
pip install --upgrade flash-attn --no-build-isolation

In [ ]:
pip install --upgrade auto-gptq

In [ ]:
pip install --upgrade optimum

In [ ]:
pip install --upgrade autoawq

In [1]:
import importlib.metadata
importlib.metadata.version("transformers")

'4.37.2'

In [2]:
importlib.metadata.version("safetensors")

'0.4.2'

In [3]:
importlib.metadata.version("accelerate")

'0.27.0'

In [4]:
importlib.metadata.version("bitsandbytes")

'0.42.0'

In [5]:
importlib.metadata.version("flash-attn")

'2.5.3'

In [6]:
importlib.metadata.version("auto-gptq")

'0.6.0'

In [7]:
importlib.metadata.version("optimum")

'1.16.2'

In [8]:
importlib.metadata.version("autoawq")

'0.1.8'

## Models

### mistralai/Mixtral-8x7B-v0.1

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, load_in_4bit=True, device_map="auto", torch_dtype="auto", attn_implementation="flash_attention_2")

### lightonai/alfred-40b-1023

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "lightonai/alfred-40b-1023"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, use_safetensors=False, load_in_4bit=True, device_map="auto", torch_dtype="auto")

### TheBloke/Mixtral-8x7B-v0.1-GPTQ

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mixtral-8x7B-v0.1-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map="auto", torch_dtype="auto", attn_implementation="flash_attention_2")

### casperhansen/mixtral-instruct-awq

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "casperhansen/mixtral-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map="auto", torch_dtype="auto", attn_implementation="flash_attention_2")

## GPU memory and disk size

In [2]:
import torch
from datetime import datetime
from IPython.display import HTML
import pickle

memory_unit = 1024*1024
total_memory = torch.cuda.get_device_properties(0).total_memory

def display_memory():
    print(torch.cuda.get_device_name(0))
    print(f"Total    : {(total_memory/memory_unit):8,.1f} MB")
    print("------------------------------")
    free_memory = torch.cuda.mem_get_info()[0]
    reserved_memory = torch.cuda.memory_reserved(0)
    used_memory = torch.cuda.memory_allocated(0)    
    max_used_memory = torch.cuda.max_memory_allocated(0)
    overhead_memory = total_memory - free_memory - reserved_memory
    print(f"Overhead : {(overhead_memory/memory_unit):8,.1f} MB - {int(overhead_memory/total_memory*100):3} %")
    print(f"Reserved : {(reserved_memory/memory_unit):8,.1f} MB - {int(reserved_memory/total_memory*100):3} %")
    print(f"Free     : {(free_memory/memory_unit):8,.1f} MB - {int(free_memory/total_memory*100):3} %")
    print("------------------------------")
    print(f"Used     : {(used_memory/memory_unit):8,.1f} MB - {int(used_memory/total_memory*100):3} %")
    print(f"Max used : {(max_used_memory/memory_unit):8,.1f} MB - {int(max_used_memory/total_memory*100):3} %")
    
def display_memory_summary():
    print(torch.cuda.memory_summary())
    
def release_cached_memory():
    torch.cuda.empty_cache()
    
def reset_peak_memory_stats():
    torch.cuda.reset_peak_memory_stats()

def record_memory_history(enabled):
    torch.cuda.memory._record_memory_history(enabled=enabled)
    
def dump_memory_snapshot():
    filename_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"memory_snapshot_{filename_datetime}.pickle"
    s = torch.cuda.memory._snapshot(0)
    with open(filename, "wb") as f:
        pickle.dump(s, f)
    print(f"Dumped memory snapshot to file: {filename}")

# https://zdevito.github.io/2022/08/16/memory-snapshots.html
# https://zdevito.github.io/2022/12/09/memory-traces.html

def display_memory_snapshot():
    url = "https://pytorch.org/memory_viz"
    return HTML(f"Call dump_memory_snapshot(), <a href='{url}' target='_blank'>click here to open Pytorch memory viz</a>, then drag and drop the snapshot file")
    
display_memory()

NVIDIA RTX A6000
Total    : 48,676.8 MB
------------------------------
Overhead :    312.0 MB -   0 %
Reserved : 23,252.0 MB -  47 %
Free     : 25,112.8 MB -  51 %
------------------------------
Used     : 22,889.9 MB -  47 %
Max used : 23,905.9 MB -  49 %


In [3]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.utils.hub import cached_file

memory_unit_mb = 1024*1024
memory_unit_gb = 1024*1024*1024

def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

def get_model_path_and_size_on_disk(pretrained_model_id):    
    model_config_file = cached_file(pretrained_model_id, "config.json", local_files_only=True)
    model_directory = os.path.dirname(os.path.dirname(model_config_file))    
    total_size = get_directory_size(model_directory)
    return model_directory,total_size

print(f"Model {model_name} size on disk:")
path,size = get_model_path_and_size_on_disk(model_name)
print(f"- model files size   : {(size/memory_unit_gb):.2f} GB")
print(f"- stored in directory: {path}")

Model lightonai/alfred-40b-1023 size on disk:
- model files size   : 155.86 GB
- stored in directory: /home/.cache/huggingface/hub/models--lightonai--alfred-40b-1023/snapshots


## Perplexity test

In [4]:
with open("/home/hftoken", 'r') as file:
    myhftoken = file.read().strip()

In [12]:
from datasets import load_dataset

#dataset_name_fr = "frenchtext/banque-fr-2311"
#dataset_fr = load_dataset(dataset_name_fr, token=myhftoken)

#dataset_name_en = "frenchtext/bank-en-2401"
#dataset_en = load_dataset(dataset_name_en, token=myhftoken)

dataset_name_de = "frenchtext/bank-de-2401"
dataset_de = load_dataset(dataset_name_de, token=myhftoken)

#dataset_name_es = "frenchtext/bank-es-2401"
#dataset_es = load_dataset(dataset_name_es, token=myhftoken)

dataset_name = dataset_name_de
split = "valid"
dataset = dataset_de[split]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

In [6]:
def get_dataset_batches(dataset, batch_size=32):
    filtered_dataset = dataset.filter(lambda example: example["Words"]>15)
    sorted_dataset = filtered_dataset.sort("Words",reverse=True)
    
    dataset_length = len(sorted_dataset)
    for start_idx in range(0, dataset_length, batch_size):
        end_idx = min(start_idx + batch_size, dataset_length)
        yield sorted_dataset[start_idx:end_idx]

def get_encoding_offsets(encoding):
    start_index = encoding.offsets[0][0]
    end_index = encoding.offsets[-1][1]
    if end_index==0: end_index = -1
    return (start_index, end_index)

def encode_dataset_batch(tokenizer, dataset_batch, stride=256):
    encodings = tokenizer(text = dataset_batch["Text"], add_special_tokens=True, 
                      padding="longest", truncation=True, return_overflowing_tokens=True, stride=stride,
                      # 2020: https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html#tensor-core-shape
                      # However now in 2023, this is less and less true, newer drivers and cuda versions are smarter about this and will be able to use tensorcores even without this aligned padding
                      pad_to_multiple_of=16, return_tensors="pt")

    encodings["overflow_to_sample_uri"] = list(map(lambda sample_id: dataset_batch["Uri"][sample_id.item()], encodings["overflow_to_sample_mapping"]))
    encodings["overflow_to_sample_offset"] = list(map(get_encoding_offsets, encodings.encodings))
    
    return encodings

def get_encodings_batches(tokenizer, dataset, batch_size=32, stride=256):
    for dataset_batch in get_dataset_batches(dataset, batch_size):
        encodings = encode_dataset_batch(tokenizer, dataset_batch, stride)
        
        encodings_length = len(encodings.encodings)
        for start_idx in range(0, encodings_length, batch_size):
            end_idx = min(start_idx + batch_size, encodings_length)
            yield {key: encodings[key][start_idx:end_idx] for key in encodings.data.keys()}

In [7]:
import torch.nn.functional as F

class PPLu():
    
    def __init__(self, dataset_iterator, tokenizer, device):
        if hasattr(tokenizer,"vocab"):
            self.vocab_size = len(tokenizer.vocab)
        else:
            self.vocab_size = tokenizer.vocab_size
        dataset_token_id_counts = torch.zeros(self.vocab_size+1, dtype=torch.int64)
        dataset_tokens_count = 0
        
        for idx,dataset_batch in enumerate(dataset_iterator):
            encodings = tokenizer(text = dataset_batch["Text"], add_special_tokens=True, padding="longest", return_tensors="pt")
            
            # Padding tokens should be ignored: count them as token_id=vocabulary_size
            token_ids = encodings.input_ids*encodings.attention_mask + self.vocab_size*(1-encodings.attention_mask)
            
            token_id_counts = torch.bincount(token_ids.view(-1), minlength=self.vocab_size+1)
            tokens_count = encodings.attention_mask.sum()

            dataset_token_id_counts += token_id_counts
            dataset_tokens_count += tokens_count
            if idx%100==9: print(f"... {dataset_tokens_count:,} tokens")
        
        # Then discard the tokens count for token_id=vocabulary_size
        self.token_id_probs =  (dataset_token_id_counts[:-1] / dataset_tokens_count).unsqueeze(1).to(device)
        self.perplexity_loss = torch.nn.CrossEntropyLoss(ignore_index=-100, reduction="none")
        print(f"Done: {dataset_tokens_count:,} tokens")

    def __call__(self, input_ids, attention_mask, output_logits):
        # Next-token prediction: shift prediction scores and input ids by one
        logits = output_logits[:, :-1, :].permute(0, 2, 1).contiguous()
        labels = input_ids[:, 1:].contiguous()
        labels_to_ignore = attention_mask[:, 1:]

        # Number of tokens predicted, ignoring padding tokens
        predicted_tokens_count = labels_to_ignore.sum(dim=1)
        
        # Cross entropy loss (ignore_index=-100)
        labels_for_crossentropy = labels*labels_to_ignore -100*(1-labels_to_ignore)
        batch_perplexity_losses = (1/predicted_tokens_count)*self.perplexity_loss(logits, labels_for_crossentropy).sum(1)
        
        # Unigram probability loss
        labels_probs = F.embedding(labels, self.token_id_probs).squeeze()
        # prob = 1 for padding tokens => log prob = 0, ignored in the sum below
        labels_probs = labels_probs*labels_to_ignore + (1-labels_to_ignore) 
        batch_unigram_losses = -(1/predicted_tokens_count)*torch.log(labels_probs).sum(dim=1)
        
        # Unigram-nomralized perplexities
        perplexities = torch.exp(batch_perplexity_losses)
        unigram_normalized_perplexities = torch.exp(batch_perplexity_losses - batch_unigram_losses)
        
        return predicted_tokens_count, batch_perplexity_losses, batch_unigram_losses, perplexities, unigram_normalized_perplexities

class NormalizedPerplexityLogger:
    def __init__(self, dataset_name, split, model_name):
        self.filename = f"{dataset_name.replace('/','_')}_{split}_{model_name.replace('/','_')}_pplu.csv"
        self.file = open(self.filename, 'w')
        
    def log_batch(self, ppl, pplu, uri, span):
        self.file.write(f"{ppl},{pplu},{uri},{span}\n")

In [14]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

#tokenizer.model_max_length = int(min(tokenizer.model_max_length, model.config.max_position_embeddings))
# Align to memory limit of RTX 4090
if tokenizer.model_max_length>8192:
    tokenizer.model_max_length = 8192
    
print(f"Computing perplexity on dataset {dataset_name}:{split} for {model_name}")
print(f"- model vocabulary: {len(tokenizer.vocab)}")
print(f"- model sequence length: {int(tokenizer.model_max_length)}")
print(f"- model torch dtype: {model.dtype}")

Computing perplexity on dataset frenchtext/bank-de-2401:valid for lightonai/alfred-40b-1023
- model vocabulary: 65024
- model sequence length: 8192
- model torch dtype: torch.bfloat16


In [15]:
%%time
pplu_loss = PPLu(get_dataset_batches(dataset), tokenizer, model.device)

... 6,062,824 tokens
Done: 9,719,122 tokens
CPU times: user 1min 7s, sys: 1.39 s, total: 1min 9s
Wall time: 24.9 s


In [16]:
batch_size = 6
#batch_size = 4
stride = 256

print(f"- dataset examples: {len(dataset)}")
print(f"- batch_size={batch_size}, stride={stride}")

- dataset examples: 2977
- batch_size=6, stride=256


In [17]:
%%time
import math

logger = NormalizedPerplexityLogger(dataset_name, split, model_name)

def display_perplexities(pred_tokens_count, ppl_losses, unigram_losses):        
    pt_pred_tokens_count = torch.Tensor(pred_tokens_count)
    total_pred_tokens_count = pt_pred_tokens_count.sum().item()
    
    pt_ppl_losses = torch.Tensor(ppl_losses)
    pt_unigram_losses = torch.Tensor(unigram_losses)    
    pt_pplu_losses = pt_ppl_losses - pt_unigram_losses

    ppl = math.exp((pt_ppl_losses*pt_pred_tokens_count).sum().item() / total_pred_tokens_count)
    pplu = math.exp((pt_pplu_losses*pt_pred_tokens_count).sum().item() / total_pred_tokens_count)

    print(f"-> perplexity = {ppl:.3f}")
    print(f"-> unigram-normalized perplexity = {pplu*1000:.3f} (x1000)")
    
pred_tokens_count = [] 
ppl_losses = []   
unigram_losses = [] 
for idx,encodings_batch in enumerate(get_encodings_batches(tokenizer, dataset, batch_size=batch_size, stride=stride)):
    with torch.no_grad():
        # predict next token
        inputs = encodings_batch["input_ids"].to(model.device)
        attention_mask = encodings_batch["attention_mask"].to(model.device)
        outputs = model(input_ids=inputs, attention_mask=attention_mask, use_cache=False, output_attentions=False, output_hidden_states=False)

        batch_pred_tokens_count, batch_ppl_losses, batch_unigram_losses, batch_ppl, batch_pplu = pplu_loss(inputs, attention_mask, outputs.logits)
        
        pred_tokens_count.extend(batch_pred_tokens_count.tolist())
        ppl_losses.extend(batch_ppl_losses.tolist())
        unigram_losses.extend(batch_unigram_losses.tolist())

    for ppl,pplu,uri,span in zip(batch_ppl.tolist(), batch_pplu.tolist(), encodings_batch["overflow_to_sample_uri"], encodings_batch["overflow_to_sample_offset"]):
        logger.log_batch(ppl, pplu, uri, span)

    if idx%10 == 0:
        print(f"{(idx+1)*batch_size} encodings processed")
        display_perplexities(pred_tokens_count, ppl_losses, unigram_losses)

print(f"FINAL RESULT: {(idx+1)*batch_size} encodings processed")
display_perplexities(pred_tokens_count, ppl_losses, unigram_losses)

6 encodings processed
-> perplexity = 3.166
-> unigram-normalized perplexity = 3.865 (x1000)
66 encodings processed
-> perplexity = 3.373
-> unigram-normalized perplexity = 3.787 (x1000)
126 encodings processed
-> perplexity = 3.459
-> unigram-normalized perplexity = 3.857 (x1000)
186 encodings processed
-> perplexity = 3.573
-> unigram-normalized perplexity = 3.757 (x1000)
246 encodings processed
-> perplexity = 3.570
-> unigram-normalized perplexity = 3.671 (x1000)
306 encodings processed
-> perplexity = 3.548
-> unigram-normalized perplexity = 3.654 (x1000)
366 encodings processed
-> perplexity = 3.547
-> unigram-normalized perplexity = 3.706 (x1000)
426 encodings processed
-> perplexity = 3.552
-> unigram-normalized perplexity = 3.754 (x1000)
486 encodings processed
-> perplexity = 3.590
-> unigram-normalized perplexity = 3.733 (x1000)
546 encodings processed
-> perplexity = 3.632
-> unigram-normalized perplexity = 3.774 (x1000)
606 encodings processed
-> perplexity = 3.667
-> unig

In [13]:
display_memory()

NVIDIA RTX A6000
Total    : 48,676.8 MB
------------------------------
Overhead :    328.0 MB -   0 %
Reserved : 47,846.0 MB -  98 %
Free     :    502.8 MB -   1 %
------------------------------
Used     : 22,959.0 MB -  47 %
Max used : 41,307.9 MB -  84 %


## Benchmark results

Computing perplexity on dataset frenchtext/bank-en-2401:valid for lightonai/alfred-40b-1023
- model vocabulary: 65024
- model sequence length: 8192
- model torch dtype: torch.bfloat16
- dataset examples: 2555
- batch_size=4, stride=256
- 9,243,621 tokens in 14 sec
- perplexity = 4.690
- unigram-normalized perplexity = 6.273 (x1000)

2 h 22 min

Computing perplexity on dataset frenchtext/banque-fr-2311:valid for lightonai/alfred-40b-1023
- model vocabulary: 65024
- model sequence length: 8192
- model torch dtype: torch.bfloat16
- dataset examples: 8522
- batch_size=4, stride=256- 13,622,486 tokens in 14 sec

- perplexity = 3.- 5
> unigram-normalized perplexity = 4.098 (x100 0)

3h 17min